<a href="https://colab.research.google.com/github/areeshaj3109/Scrapes/blob/main/Kidzy-Scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cloudscraper
!pip install selenium
!apt update
!apt install chromium-chromedriver


from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import cloudscraper
import requests
from bs4 import BeautifulSoup

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

scraper = cloudscraper.create_scraper()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
58 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as

In [ ]:
def get_product_info(url):
  name = ''
  price = ''
  sku = ''
  cats = ''
  desc = ''
  img = []

  response = scraper.get(url)

  if response.status_code == 200:
    response = response.content
    soup = BeautifulSoup(response, 'html.parser')

  name_tag = soup.find('h1', class_ = 'product_title entry-title')
  if name_tag:
    name = name_tag.text.strip()
    # print(name)

  price_tag = soup.find('span', class_ = 'woocommerce-Price-amount amount')
  if price_tag:
    price = price_tag.text.strip()
    # print(price)

  sku_tag = soup.find('span', class_ = 'sku_wrapper')
  if sku_tag:
    sku = sku_tag.text[5:].strip()
    # print(sku)

  cat_tag = soup.find('span', class_ = 'posted_in')
  if cat_tag:
    cat = cat_tag.text[11:].strip()
    # print(cat)

  desc_tag = soup.find('div', class_ = 'woocommerce-product-details__short-description')
  if desc_tag:
    desc = desc_tag.text.strip().replace('\n', '. ')
    # print(desc)

  gallery = soup.find('div', class_ = 'veb-variation-gallery-thumbnail-slider veb-variation-gallery-thumbnail-columns-4')
  if gallery:
    images = gallery.find_all('div', class_ = 'iwp-gallery-thumbnail-image')
    if images:
      for i in images:
        img.append(i.find('img')['src'])
      # print(img)
    else:
      images = soup.find('div', class_ = 'iwp-single-gallery-image-container')
      if images:
        img.append(images.find('img')['src'])

  return {
          'Name': name,
          'Price': price,
          'SKU': sku,
          'Category': cat,
          'Description': desc,
          'Image Links': img
  }

In [ ]:
url = 'https://kidzy.qa/shop/?lang=en'

def get_product_urls(url):
  product_urls = []
  response = scraper.get(url)

  if response.status_code == 200:
    response = response.content
    soup = BeautifulSoup(response, 'html.parser')

  all_products = soup.find('ul', class_ = 'products columns-4')

  product_url = all_products.find_all('div', class_ = 'mf-product-thumbnail')

  for i in product_url:
    product_urls.append(i.find('a').get('href'))

  return product_urls

In [ ]:
def get_page_url(url):
  page_urls = [url]
  start = 0
  end = 0

  response = scraper.get(url)

  if response.status_code == 200:
    response = response.content
    soup = BeautifulSoup(response, 'html.parser')

  pages = soup.find('ul', class_ = 'page-numbers')
  if pages:
    page_numbers = []

    for li in pages.find_all('li'):
      span = li.find('span')
      a = li.find('a')

      if span:
        page_numbers.append(span.text.strip())
      elif a:
        page_numbers.append(a.text.strip())
  else:
    print("No pagination found.")

  start = int(page_numbers[0])
  end = int(page_numbers[-2])

  for i in range(start + 1, end + 1):
    num = str(i)
    page_urls.append('https://kidzy.qa/shop/page/' + num + '/?lang=en')
  return page_urls

In [ ]:
page_urls = get_page_url('https://kidzy.qa/shop/?lang=en')

In [ ]:
c = 1
product_data = []
for i in page_urls:
  print(f'{c} / {len(page_urls)}')
  c += 1
  product_urls = get_product_urls(i)
  for product_url in product_urls:
    print(product_url)
    prod_info = get_product_info(product_url)
    if prod_info is not None:
      product_data.append(prod_info)
    else:
      print("prod_info is None")

df = pd.DataFrame(product_data)

https://kidzy.qa/product/galt-animal-pottery/?lang=en
https://kidzy.qa/product/2-in-1-games-ludo-snakes-ladders/?lang=en
https://kidzy.qa/product/moon-jeep-gladiator-pink/?lang=en
https://kidzy.qa/product/rollplay-nighthawk-drift-car-12v/?lang=en
https://kidzy.qa/product/hellokitty-doll-14/?lang=en
https://kidzy.qa/product/little-tikes-cozy-coupeladybird/?lang=en
https://kidzy.qa/product/hellokitty-doll/?lang=en
https://kidzy.qa/product/disney-princess-style-collection-light-up-sounds/?lang=en
https://kidzy.qa/product/little-tikes-shop-n-learn-smart-checkout/?lang=en
https://kidzy.qa/product/x-shot-hyper-gel-medium-blaster-10000gellets/?lang=en
https://kidzy.qa/product/mould-paint-space/?lang=en
https://kidzy.qa/product/water-magic-unicorns/?lang=en
https://kidzy.qa/product/water-magic-safari/?lang=en
https://kidzy.qa/product/little-tikes-play-n-scoot-pirate-ship/?lang=en
https://kidzy.qa/product/little-tikes-the-first-self-checkout-stand/?lang=en
https://kidzy.qa/product/water-magic-u

In [ ]:
df